In [2]:
%logstop
%logstart -rtq ~/.logs/dw.py append
%matplotlib inline
import matplotlib
import seaborn as sns
sns.set()
matplotlib.rcParams['figure.dpi'] = 144

In [3]:
from static_grader import grader

# DW Miniproject
## Introduction

The objective of this miniproject is to exercise your ability to wrangle tabular data set and aggregate large data sets into meaningful summary statistics. We'll work with the same medical data used in the `pw` miniproject but leverage the power of Pandas to more efficiently represent and act on our data.

## Downloading the data

We first need to download the data we'll be using from Amazon S3:

In [4]:
!mkdir dw-data
!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/201701scripts_sample.csv.gz -nc -P ./dw-data/
!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/201606scripts_sample.csv.gz -nc -P ./dw-data/
!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/practices.csv.gz -nc -P ./dw-data/
!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/chem.csv.gz -nc -P ./dw-data/

mkdir: cannot create directory ‘dw-data’: File exists
File ‘./dw-data/201701scripts_sample.csv.gz’ already there; not retrieving.

File ‘./dw-data/201606scripts_sample.csv.gz’ already there; not retrieving.

File ‘./dw-data/practices.csv.gz’ already there; not retrieving.

File ‘./dw-data/chem.csv.gz’ already there; not retrieving.



## Loading the data

Similar to the `PW` miniproject, the first step is to read in the data. The data files are stored as compressed CSV files. You can load the data into a Pandas DataFrame by making use of the `gzip` package to decompress the files and Panda's `read_csv` methods to parse the data into a DataFrame. You may want to check the Pandas documentation for parsing [CSV](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) files for reference.

For a description of the data set please, refer to the [PW miniproject](./pw.ipynb). **Note that all questions make use of the 2017 data only, except for Question 5 which makes use of both the 2017 and 2016 data.**

In [5]:
import pandas as pd
import numpy as np
import gzip

In [6]:

# load the 2017 data
with gzip.open('./dw-data/201701scripts_sample.csv.gz', 'rb') as f:
    scripts = pd.read_csv(f)

scripts.head()

,practice,bnf_code,bnf_name,items,nic,act_cost,quantity
0,N85639,0106020C0,Bisacodyl_Tab E/C 5mg,1,0.39,0.47,12
1,N85639,0106040M0,Movicol Plain_Paed Pdr Sach 6.9g,1,4.38,4.07,30
2,N85639,0301011R0,Salbutamol_Inha 100mcg (200 D) CFF,1,1.50,1.40,1
3,N85639,0304010G0,Chlorphenamine Mal_Oral Soln 2mg/5ml,1,2.62,2.44,150
4,N85639,0401020K0,Diazepam_Tab 2mg,1,0.16,0.26,6


In [7]:
col_names=[ 'code', 'name', 'addr_1', 'addr_2', 'borough', 'village', 'post_code']
with gzip.open('./dw-data/practices.csv.gz', 'rb') as f:    
    practices = pd.read_csv(f, names=col_names)
    
practices.head()

,code,name,addr_1,addr_2,borough,village,post_code
0,A81001,THE DENSHAM SURGERY,THE HEALTH CENTRE,LAWSON STREET,STOCKTON ON TEES,CLEVELAND,TS18 1HU
1,A81002,QUEENS PARK MEDICAL CENTRE,QUEENS PARK MEDICAL CTR,FARRER STREET,STOCKTON ON TEES,CLEVELAND,TS18 2AW
2,A81003,VICTORIA MEDICAL PRACTICE,THE HEALTH CENTRE,VICTORIA ROAD,HARTLEPOOL,CLEVELAND,TS26 8DB
3,A81004,WOODLANDS ROAD SURGERY,6 WOODLANDS ROAD,NaN,MIDDLESBROUGH,CLEVELAND,TS1 3BE
4,A81005,SPRINGWOOD SURGERY,SPRINGWOOD SURGERY,RECTORY LANE,GUISBOROUGH,NaN,TS14 7DJ


In [8]:
with gzip.open('./dw-data/chem.csv.gz', 'rb') as f:
    chem = pd.read_csv(f)
chem.head()

,CHEM SUB,NAME
0,0101010A0,Alexitol Sodium
1,0101010B0,Almasilate
2,0101010C0,Aluminium Hydroxide
3,0101010D0,Aluminium Hydroxide With Magnesium
4,0101010E0,Hydrotalcite


Now that we've loaded in the data, let's first replicate our results from the `PW` miniproject. Note that we are now working with a larger data set so the answers will be different than in the `PW` miniproject even if the analysis is the same.

## Question 1: summary_statistics

In the `PW` miniproject we first calculated the total, mean, standard deviation, and quartile statistics of the `'items'`, `'quantity'`', `'nic'`, and `'act_cost'` fields. To do this we had to write some functions to calculate the statistics and apply the functions to our data structure. The DataFrame has a `describe` method that will calculate most (not all) of these things for us.

Submit the summary statistics to the grader as a list of tuples: [('act_cost', (total, mean, std, q25, median, q75)), ...]

In [37]:
def dscrb(key):
    k = scripts[key].describe()
    return (scripts[key].sum(),k[1],k[2],k[4],k[5],k[6])
dscrb('items')

(8888304, 9.133135976111625, 29.204198282803603, 1.0, 2.0, 6.0)

In [38]:
summary_stats = [('items', dscrb('items')), ('quantity', dscrb('quantity')), ('nic', dscrb('nic')), ('act_cost', dscrb('act_cost'))]

In [39]:
grader.score.dw__summary_statistics(summary_stats)

Your score:  1.0


## Question 2: most_common_item

We can also easily compute summary statistics on groups within the data. In the `pw` miniproject we had to explicitly construct the groups based on the values of a particular field. Pandas will handle that for us via the `groupby` method. This process is [detailed in the Pandas documentation](https://pandas.pydata.org/pandas-docs/stable/groupby.html).

Use `groupby` to calculate the total number of items dispensed for each `'bnf_name'`. Find the item with the highest total and return the result as `[(bnf_name, total)]`.

In [40]:
by_names = scripts.groupby(['bnf_name'])['items'].sum()

In [41]:
most_common_item = [(by_names.idxmax(), by_names.max())]

In [42]:
grader.score.dw__most_common_item(most_common_item)

Your score:  1.0


## Question 3: items_by_region

Now let's find the most common item by post code. The post code information is in the `practices` DataFrame, and we'll need to `merge` it into the `scripts` DataFrame. Pandas provides [extensive documentation](https://pandas.pydata.org/pandas-docs/stable/merging.html) with diagrammed examples on different methods and approaches for joining data. The `merge` method is only one of many possible options.

Return your results as a list of tuples `(post code, item name, amount dispensed as % of total)`. Sort your results ascending alphabetically by post code and take only results from the first 100 post codes.

**NOTE:** Some practices have multiple postal codes associated with them. Use the alphabetically first postal code. Note some postal codes may have multiple `'bnf_name'` with the same prescription rate for the maximum. In this case, take the alphabetically first `'bnf_name'` (as in the PW miniproject).

In [43]:
#find all unique practice
unique_practice = practices.sort_values('post_code').groupby('code').first().reset_index()
unique_practice.head()

,code,name,addr_1,addr_2,borough,village,post_code
0,A81001,THE DENSHAM SURGERY,THE HEALTH CENTRE,LAWSON STREET,STOCKTON ON TEES,CLEVELAND,TS18 1HU
1,A81002,QUEENS PARK MEDICAL CENTRE,QUEENS PARK MEDICAL CTR,FARRER STREET,STOCKTON ON TEES,CLEVELAND,TS18 2AW
2,A81003,VICTORIA MEDICAL PRACTICE,MCKENZIE HOUSE,17 KENDAL ROAD,HARTLEPOOL,CLEVELAND,TS25 1QU
3,A81004,WOODLANDS ROAD SURGERY,6 WOODLANDS ROAD,ACKLAM,MIDDLESBROUGH,CLEVELAND,TS1 3BE
4,A81005,SPRINGWOOD SURGERY,SPRINGWOOD SURGERY,RECTORY LANE,GUISBOROUGH,NaN,TS14 7DJ


In [44]:
#join them with scripts
joined = scripts.merge(unique_practice[['code','post_code']], left_on='practice', right_on='code', how = 'left')
joined.head()

,practice,bnf_code,bnf_name,items,nic,act_cost,quantity,code,post_code
0,N85639,0106020C0,Bisacodyl_Tab E/C 5mg,1,0.39,0.47,12,N85639,CH44 5UF
1,N85639,0106040M0,Movicol Plain_Paed Pdr Sach 6.9g,1,4.38,4.07,30,N85639,CH44 5UF
2,N85639,0301011R0,Salbutamol_Inha 100mcg (200 D) CFF,1,1.50,1.40,1,N85639,CH44 5UF
3,N85639,0304010G0,Chlorphenamine Mal_Oral Soln 2mg/5ml,1,2.62,2.44,150,N85639,CH44 5UF
4,N85639,0401020K0,Diazepam_Tab 2mg,1,0.16,0.26,6,N85639,CH44 5UF


In [45]:
#find total of items for each bnf name and post code
post_items_totals = joined.groupby(['post_code','bnf_name'])['items'].sum().reset_index()
post_items_totals.head()

,post_code,bnf_name,items
0,B11 4BW,3m Health Care_Cavilon Durable Barrier C,7
1,B11 4BW,3m Health Care_Cavilon No Sting Barrier,2
2,B11 4BW,Abasaglar KwikPen_100u/ml 3ml Pf Pen,2
3,B11 4BW,Abidec_Dps,63
4,B11 4BW,Able Spacer + Sml/Med Mask,1


In [46]:
#location of max
idx = post_items_totals.groupby('post_code')['items'].idxmax()
idx.head()

post_code
B11 4BW    1628
B12 9LP    3323
B18 7AL    5293
B21 9RY    6924
B23 6DJ    9096
Name: items, dtype: int64

In [47]:
max_items = post_items_totals.loc[idx].set_index('post_code')
max_items.head()

,bnf_name,items
post_code,,
B11 4BW,Salbutamol_Inha 100mcg (200 D) CFF,706
B12 9LP,Paracet_Tab 500mg,425
B18 7AL,Salbutamol_Inha 100mcg (200 D) CFF,556
B21 9RY,Metformin HCl_Tab 500mg,1033
B23 6DJ,Lansoprazole_Cap 30mg (E/C Gran),599


In [48]:
max_items['items_ratio'] = (max_items['items']/post_items_totals.groupby('post_code')['items'].sum())
max_items.head()

,bnf_name,items,items_ratio
post_code,,,
B11 4BW,Salbutamol_Inha 100mcg (200 D) CFF,706,0.031059
B12 9LP,Paracet_Tab 500mg,425,0.024893
B18 7AL,Salbutamol_Inha 100mcg (200 D) CFF,556,0.027111
B21 9RY,Metformin HCl_Tab 500mg,1033,0.033294
B23 6DJ,Lansoprazole_Cap 30mg (E/C Gran),599,0.021384


In [49]:
result = max_items.reset_index().head(100)[['post_code','bnf_name','items_ratio']].values.tolist()
result[:3]


[['B11 4BW', 'Salbutamol_Inha 100mcg (200 D) CFF', 0.031058906339360346],
 ['B12 9LP', 'Paracet_Tab 500mg', 0.02489310607391788],
 ['B18 7AL', 'Salbutamol_Inha 100mcg (200 D) CFF', 0.027111371172225472]]

In [50]:
items_by_region =[(tuple(item)) for item in result]
items_by_region[:3]
#items_by_region = [("B11 4BW", "Salbutamol_Inha 100mcg (200 D) CFF", 0.0310589063)] * 100

[('B11 4BW', 'Salbutamol_Inha 100mcg (200 D) CFF', 0.031058906339360346),
 ('B12 9LP', 'Paracet_Tab 500mg', 0.02489310607391788),
 ('B18 7AL', 'Salbutamol_Inha 100mcg (200 D) CFF', 0.027111371172225472)]

In [51]:
grader.score.dw__items_by_region(items_by_region)

Your score:  1.0


## Question 4: script_anomalies

Drug abuse is a source of human and monetary costs in health care. A first step in identifying practitioners that enable drug abuse is to look for practices where commonly abused drugs are prescribed unusually often. Let's try to find practices that prescribe an unusually high amount of opioids. The opioids we'll look for are given in the list below.

In [52]:
opioids = ['morphine', 'oxycodone', 'methadone', 'fentanyl', 'pethidine', 'buprenorphine', 'propoxyphene', 'codeine']

These are generic names for drugs, not brand names. Generic drug names can be found using the `'bnf_code'` field in `scripts` along with the `chem` table.. Use the list of opioids provided above along with these fields to make a new field in the `scripts` data that flags whether the row corresponds with a opioid prescription.

In [53]:
scripts.head()


,practice,bnf_code,bnf_name,items,nic,act_cost,quantity
0,N85639,0106020C0,Bisacodyl_Tab E/C 5mg,1,0.39,0.47,12
1,N85639,0106040M0,Movicol Plain_Paed Pdr Sach 6.9g,1,4.38,4.07,30
2,N85639,0301011R0,Salbutamol_Inha 100mcg (200 D) CFF,1,1.50,1.40,1
3,N85639,0304010G0,Chlorphenamine Mal_Oral Soln 2mg/5ml,1,2.62,2.44,150
4,N85639,0401020K0,Diazepam_Tab 2mg,1,0.16,0.26,6


In [54]:
chem.head()


,CHEM SUB,NAME
0,0101010A0,Alexitol Sodium
1,0101010B0,Almasilate
2,0101010C0,Aluminium Hydroxide
3,0101010D0,Aluminium Hydroxide With Magnesium
4,0101010E0,Hydrotalcite


In [55]:
opioid_codes = chem.loc[chem['NAME'].str.contains('|'.join(opioids), case=False)]['CHEM SUB'].tolist()
opioid_codes

['0104020D0',
 '0104020N0',
 '0309010C0',
 '0309010N0',
 '0309010S0',
 '0309020AC',
 '0407010F0',
 '0407010M0',
 '0407010N0',
 '0407010R0',
 '0407010T0',
 '0407010V0',
 '0407020AD',
 '0407020AE',
 '0407020AF',
 '0407020A0',
 '0407020B0',
 '0407020C0',
 '0407020E0',
 '0407020G0',
 '0407020K0',
 '0407020M0',
 '0407020N0',
 '0407020P0',
 '0407020Q0',
 '0407020V0',
 '0407020Z0',
 '040702010',
 '040702020',
 '0409010A0',
 '0410030A0',
 '0410030C0',
 '0704050G0',
 '1501043F0',
 '1502010Z0']

In [93]:
scripts['opioid'] = scripts['bnf_code'].isin(opioid_codes)
scripts.head()

,practice,bnf_code,bnf_name,items,nic,act_cost,quantity,opioid
0,N85639,0106020C0,Bisacodyl_Tab E/C 5mg,1,0.39,0.47,12,False
1,N85639,0106040M0,Movicol Plain_Paed Pdr Sach 6.9g,1,4.38,4.07,30,False
2,N85639,0301011R0,Salbutamol_Inha 100mcg (200 D) CFF,1,1.50,1.40,1,False
3,N85639,0304010G0,Chlorphenamine Mal_Oral Soln 2mg/5ml,1,2.62,2.44,150,False
4,N85639,0401020K0,Diazepam_Tab 2mg,1,0.16,0.26,6,False


Now for each practice calculate the proportion of its prescriptions containing opioids.

**Hint:** Consider the following list: `[0, 1, 1, 0, 0, 0]`. What proportion of the entries are 1s? What is the mean value?

In [57]:

opioids_per_practice = scripts.groupby('practice')['opioid'].mean().rename('prop')
opioids_per_practice.head()

practice
A81005    0.033179
A81007    0.043329
A81011    0.046556
A81012    0.042793
A81017    0.038140
Name: prop, dtype: float64

How do these proportions compare to the overall opioid prescription rate? Subtract off the proportion of all prescriptions that are opioids from each practice's proportion.

In [94]:
overall_rate = scripts['opioid'].mean()
relative_opioids_per_practice = (opioids_per_practice - overall_rate).rename('rel')
relative_opioids_per_practice.head()

practice
A81005   -0.002624
A81007    0.007526
A81011    0.010753
A81012    0.006990
A81017    0.002337
Name: rel, dtype: float64

Now that we know the difference between each practice's opioid prescription rate and the overall rate, we can identify which practices prescribe opioids at above average or below average rates. However, are the differences from the overall rate important or just random deviations? In other words, are the differences from the overall rate big or small?

To answer this question we have to quantify the difference we would typically expect between a given practice's opioid prescription rate and the overall rate. This quantity is called the **standard error**, and is related to the **standard deviation**, $\sigma$. The standard error in this case is

$$ \frac{\sigma}{\sqrt{n}} $$

where $n$ is the number of prescriptions each practice made. Calculate the standard error for each practice. Then divide `relative_opioids_per_practice` by the standard errors. We'll call the final result `opioid_scores`.

In [69]:
n = scripts['practice'].value_counts().astype(int).rename('number_of_scripts')
n.head()

N83028    2844
L83100    2797
D81043    2459
B81008    2396
B81026    2347
Name: number_of_scripts, dtype: int64

In [71]:
standard_error_per_practice = (scripts['opioid'].var()/n)**0.5
standard_error_per_practice.head()


N83028    0.003484
L83100    0.003513
D81043    0.003747
B81008    0.003796
B81026    0.003835
Name: number_of_scripts, dtype: float64

In [72]:
opioid_scores = (relative_opioids_per_practice/standard_error_per_practice).rename('z-score')
opioid_scores.head()

A81005   -0.548306
A81007    1.544557
A81011    2.291795
A81012    1.373060
A81017    0.583168
Name: z-score, dtype: float64

The quantity we have calculated in `opioid_scores` is called a **z-score**:

$$ \frac{\bar{X} - \mu}{\sqrt{\sigma^2/n}} $$

Here $\bar{X}$ corresponds with the proportion for each practice, $\mu$ corresponds with the proportion across all practices, $\sigma^2$ corresponds with the variance of the proportion across all practices, and $n$ is the number of prescriptions made by each practice. Notice $\bar{X}$ and $n$ will be different for each practice, while $\mu$ and $\sigma$ are determined across all prescriptions, and so are the same for every z-score. The z-score is a useful statistical tool used for hypothesis testing, finding outliers, and comparing data about different types of objects or events.

Now that we've calculated this statistic, take the 100 practices with the largest z-score. Return your result as a list of tuples in the form `(practice_name, z-score, number_of_scripts)`. Sort your tuples by z-score in descending order. Note that some practice codes will correspond with multiple names. In this case, use the first match when sorting names alphabetically.

In [32]:
#mapping code of practice with name
unique_practices = practices.groupby('code')['name'].min()
unique_practices.head()


code
A81001           THE DENSHAM SURGERY
A81002    QUEENS PARK MEDICAL CENTRE
A81003     VICTORIA MEDICAL PRACTICE
A81004       BLUEBELL MEDICAL CENTRE
A81005            SPRINGWOOD SURGERY
Name: name, dtype: object

In [88]:
output= pd.concat([opioid_scores, n], axis=1)
results = output.reset_index().sort_values('z-score', ascending=False).head(100)

results.head()


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,index,z-score,number_of_scripts
714,Y01852,11.695818,7
754,Y03006,7.339043,2
772,Y03668,6.150582,60
261,G81703,5.123032,36
819,Y04997,4.958866,321


In [89]:
results['practice_name'] = results['index'].apply(lambda code: unique_practices[code])


results.head()

,index,z-score,number_of_scripts,practice_name
714,Y01852,11.695818,7,NATIONAL ENHANCED SERVICE
754,Y03006,7.339043,2,OUTREACH SERVICE NH / RH
772,Y03668,6.150582,60,BRISDOC HEALTHCARE SERVICES OOH
261,G81703,5.123032,36,H&R P C SPECIAL SCHEME
819,Y04997,4.958866,321,HMR BARDOC OOH


In [91]:
anomalies =[(tuple(item)) for item in results[['practice_name', 'z-score', 'number_of_scripts']].values.tolist()]
anomalies[:3]
#anomalies = [("NATIONAL ENHANCED SERVICE", 11.6958178629, 7)] * 100

[('NATIONAL ENHANCED SERVICE', 11.695817862936027, 7),
 ('OUTREACH SERVICE NH / RH', 7.339043019238823, 2),
 ('BRISDOC HEALTHCARE SERVICES OOH', 6.1505817490838295, 60)]

In [92]:
grader.score.dw__script_anomalies(anomalies)

Your score:  1.0


## Question 5: script_growth

Another way to identify anomalies is by comparing current data to historical data. In the case of identifying sites of drug abuse, we might compare a practice's current rate of opioid prescription to their rate 5 or 10 years ago. Unless the nature of the practice has changed, the profile of drugs they prescribe should be relatively stable. We might also want to identify trends through time for business reasons, identifying drugs that are gaining market share. That's what we'll do in this question.

We'll load in beneficiary data from 6 months earlier, June 2016, and calculate the percent growth in prescription rate from June 2016 to January 2017 for each `bnf_name`. We'll return the 50 items with largest growth and the 50 items with the largest shrinkage (i.e. negative percent growth) as a list of tuples sorted by growth rate in descending order in the format `(script_name, growth_rate, raw_2016_count)`. You'll notice that many of the 50 fastest growing items have low counts of prescriptions in 2016. Filter out any items that were prescribed less than 50 times.

In [9]:

# load the 2016 data
with gzip.open('./dw-data/201606scripts_sample.csv.gz', 'rb') as f:
    scripts16 = pd.read_csv(f)

scripts16.head()


,practice,bnf_code,bnf_name,items,nic,act_cost,quantity
0,N85638,0301011R0,Salamol_Inha 100mcg (200 D) CFF (Teva),2,2.92,2.73,2
1,N85638,0301011R0,Easyhaler_Salbutamol Sulf 200mcg (200D),1,6.63,6.15,1
2,N85638,0301020I0,Ipratrop Brom_Inh Soln 500mcg/2ml Ud,1,1.77,1.75,12
3,N85638,0301020I0,Ipratrop Brom_Inh Soln 250mcg/1ml Ud,1,4.47,4.15,20
4,N85638,0302000C0,Clenil Modulite_Inha 50mcg (200D),1,3.70,3.44,1


In [12]:
# growth = (current - past)/past
pct_growth = (scripts['bnf_name'].value_counts() / scripts16['bnf_name'].value_counts()) -1
#normalization growth - nr
norm_pct_growth= pct_growth - ((scripts['bnf_name'].count() / scripts16['bnf_name'].count()) -1)
norm_pct_growth=norm_pct_growth.sort_values(ascending=False)
norm_pct_growth.head
#95% between -2 - +2 z-score

<bound method NDFrame.head of Vensir XL_Cap 225mg                         92.000
Fludroxycortide_Tape 7.5cm x 20cm           52.000
Tamiflu_Cap 75mg                            43.000
Enstilar_Foam Aero 50mcg/0.5mg/g            27.125
Memantine HCl_Orodisper Tab 10mg S/F        27.000
                                             ...  
iMEDicare_Urine Collection Bag 2000ml          NaN
iMEDicare_Urine Collection Bag 500ml Ven       NaN
kliniderm Foam Lite Border 10cm x 10cm W       NaN
kliniderm Foam Lite Border 7.5cmx7.5cm W       NaN
kliniderm Foam Slc Heel 10cm x 17.5cm Wo       NaN
Name: bnf_name, Length: 15424, dtype: float64>

In [13]:
scripts16['bnf_name'].value_counts().head()

GlucoRX FinePoint Needles Pen Inj Screw     1532
3m Health Care_Cavilon Durable Barrier C     825
Fluclox Sod_Cap 500mg                        796
Amoxicillin_Cap 500mg                        790
Prednisolone_Tab 5mg                         786
Name: bnf_name, dtype: int64

In [19]:
output=pd.concat([norm_pct_growth.rename('pct_growth'), scripts16['bnf_name'].value_counts().rename('count')], axis=1)
output= output.dropna().sort_values('pct_growth', ascending=False).query('count>50')
output.head()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,pct_growth,count
Butec_Transdermal Patch 5mcg/hr,3.467742,62.0
Butec_Transdermal Patch 10mcg/hr,3.000000,69.0
Fostair NEXThaler_Inh 200mcg/6mcg (120D),1.430233,86.0
Pneumococcal_Vac 0.5ml Vl (23 Valent),1.269430,193.0
Spiolto Respimat_Inha2.5/2.5mcg(60D)+Dev,1.269231,52.0


In [23]:
most_extreme = pd.concat([output.head(50), output.tail(50)], axis=0).reset_index()
most_extreme

,index,pct_growth,count
0,Butec_Transdermal Patch 5mcg/hr,3.467742,62.0
1,Butec_Transdermal Patch 10mcg/hr,3.000000,69.0
2,Fostair NEXThaler_Inh 200mcg/6mcg (120D),1.430233,86.0
3,Pneumococcal_Vac 0.5ml Vl (23 Valent),1.269430,193.0
4,Spiolto Respimat_Inha2.5/2.5mcg(60D)+Dev,1.269231,52.0
5,Trulicity_Inj 1.5mg/0.5ml Pf Pen,1.185185,54.0
6,CosmoCol_Paed Oral Pdr Sach 6.9g,1.177419,62.0
7,Dulaglutide_Inj 1.5mg/0.5ml Pf Dev,0.963415,82.0
8,ViATIM_Vac D/Chamber 160u/25mcg 1ml Pfs,0.912000,125.0
9,Empagliflozin_Tab 25mg,0.896000,125.0


In [25]:
script_growth =[(tuple(item)) for item in most_extreme[['index','pct_growth','count']].values.tolist()]
script_growth[:3]

[('Butec_Transdermal Patch 5mcg/hr', 3.467741935483871, 62.0),
 ('Butec_Transdermal Patch 10mcg/hr', 3.0, 69.0),
 ('Fostair NEXThaler_Inh 200mcg/6mcg (120D)', 1.4302325581395348, 86.0)]

In [63]:
#script_growth = [("Butec_Transdermal Patch 5mcg\/hr", 3.4677419355, 62.0)] * 100

In [26]:
grader.score.dw__script_growth(script_growth)

Your score:  0.9900000000000007


## Question 6: rare_scripts

Does a practice's prescription costs originate from routine care or from reliance on rarely prescribed treatments? Commonplace treatments can carry lower costs than rare treatments because of efficiencies in large-scale production. While some specialist practices can't help but avoid prescribing rare medicines because there are no alternatives, some practices may be prescribing a unnecessary amount of brand-name products when generics are available. Let's identify practices whose costs disproportionately originate from rarely prescribed items.

First we have to identify which `'bnf_code'` are rare. To do this, find the probability $p$ of a prescription having a particular `'bnf_code'` if the `'bnf_code'` was randomly chosen from the unique options in the beneficiary data. We will call a `'bnf_code'` rare if it is prescribed at a rate less than $0.1p$.

In [9]:
p = 1 / scripts['bnf_code'].nunique()
rates = scripts['bnf_code'].value_counts() / scripts['bnf_code'].count()


#scripts['rare'] = ...

In [11]:
rare_codes = set(rates[rates < 0.1 * p].index)

In [13]:
scripts['rare'] = scripts['bnf_code'].isin(rare_codes)
scripts.head()

,practice,bnf_code,bnf_name,items,nic,act_cost,quantity,rare
0,N85639,0106020C0,Bisacodyl_Tab E/C 5mg,1,0.39,0.47,12,False
1,N85639,0106040M0,Movicol Plain_Paed Pdr Sach 6.9g,1,4.38,4.07,30,False
2,N85639,0301011R0,Salbutamol_Inha 100mcg (200 D) CFF,1,1.50,1.40,1,False
3,N85639,0304010G0,Chlorphenamine Mal_Oral Soln 2mg/5ml,1,2.62,2.44,150,False
4,N85639,0401020K0,Diazepam_Tab 2mg,1,0.16,0.26,6,False


Now for each practice, calculate the proportion of costs that originate from prescription of rare treatments (i.e. rare `'bnf_code'`). Use the `'act_cost'` field for this calculation.

In [26]:
rare_cost_prop = (scripts[scripts['rare']].groupby('practice')['act_cost'].sum() / 
                  scripts.groupby('practice')['act_cost'].sum())
rare_cost_prop = rare_cost_prop.fillna(0)
rare_cost_prop

practice
A81005    0.012017
A81007    0.008381
A81011    0.005116
A81012    0.013747
A81017    0.007359
            ...   
Y05570    0.000000
Y05583    0.000000
Y05597    0.000000
Y05660    0.000323
Y05670    0.000000
Name: act_cost, Length: 856, dtype: float64

Now we will calculate a z-score for each practice based on this proportion.
First take the difference of `rare_cost_prop` and the proportion of costs originating from rare treatments across all practices.

In [27]:
relative_rare_cost_prop = rare_cost_prop - scripts[scripts['rare']]['act_cost'].sum() / scripts['act_cost'].sum()

Now we will estimate the standard errors (i.e. the denominator of the z-score) by simply taking the standard deviation of this difference.

In [28]:
standard_errors = relative_rare_cost_prop.std()

Finally compute the z-scores. Return the practices with the top 100 z-scores in the form `(post_code, practice_name, z-score)`. Note that some practice codes will correspond with multiple names. In this case, use the first match when sorting names alphabetically.

In [30]:
rare_scores = (relative_rare_cost_prop / standard_errors).reset_index()
rare_scores.head()

,practice,act_cost
0,A81005,-0.065216
1,A81007,-0.125308
2,A81011,-0.179263
3,A81012,-0.036615
4,A81017,-0.142190


In [46]:
rare_scores['practice_name'] = rare_scores['practice'].apply(lambda code: unique_practices[code])
rare_scores = rare_scores.rename({'act_cost': 'z-score'}, axis=1)
rare_scores = rare_scores.sort_values('z-score', ascending=False).head(100)
rare_scores.head()

,practice,z-score,practice_name
765,Y03472,16.262687,CONSULTANT DIABETES TEAM
831,Y05320,15.128648,DMC COMMUNITY DERMATOLOGY RBWF
793,Y04404,7.542139,OUTPATIENTS JUBILEE HEALTH CENTRE
766,Y03484,7.287222,DMC COMMUNITY DERMATOLOGY CLINIC
794,Y04424,6.838614,DMC HEALTHCARE


In [48]:
rare_scripts = [(tuple(item)) for item in rare_scores[['practice','practice_name','z-score']].values.tolist()]
rare_scripts[:3]
#rare_scripts = [("Y03472", "CONSULTANT DIABETES TEAM", 16.2626871247)] * 100

[('Y03472', 'CONSULTANT DIABETES TEAM', 16.262687124655073),
 ('Y05320', 'DMC COMMUNITY DERMATOLOGY RBWF', 15.128648195416869),
 ('Y04404', 'OUTPATIENTS JUBILEE HEALTH CENTRE', 7.54213935610462)]

In [49]:
grader.score.dw__rare_scripts(rare_scripts)

Your score:  1.0


*Copyright &copy; 2020 The Data Incubator.  All rights reserved.*